## Procesamiento de Lenguaje Natural

*MINI-TASK \#1* 

# ***Byte-Pair-Encoding del Quijote***

### **Equipo:**

- Giottonini Herrera Enrique Alejandro
-
-
- Villalba Miranda Jesús Abraham

## **Introducción**

## Aprendiendo de nuestro dataset
Antes de armar el algoritmo de **BPE** primero explicaremos las funciones que necesitamos.

Importamos las librerias que necesitamos

In [ ]:
import re, collections

### **Vocab per words**
Explicación de la función

In [ ]:
def get_vocab(filename):
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            words = line.strip().split()
            for word in words:
                vocab[' '.join(list(word)) + ' </w>'] += 1
    return vocab

In [ ]:
vocabulario = get_vocab('corpus/extract.txt')

In [ ]:
print("Vocabulario inicial:")
for word, freq in vocabulario.items():
    print(f"{word}: {freq}")

### **Obteniendo los pares**

In [ ]:
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

In [ ]:
pairs = get_stats(vocabulario)

In [ ]:
sorted_pairs = sorted(get_stats(vocabulario).items(), key=lambda x:x[1],reverse=True)
for i in range(10):
    print(sorted_pairs[i])

In [ ]:
best = max(pairs, key=pairs.get)
print(f"El par de caracteres consecutivos más frecuentes es: {best}")

### **Merge**
La función `merge_vocab` es la encargada de actualizar el vocabulario al mezclar el par de caracteres más frecuente. 

Esta función recibe:
1. **pair**: par más frecuente en el vocabulario
2. **v_in**: el vocabulario.

La función regresa el vocabulario actualizado despues de unir los caracteres y actualizar la frequencia de los tokens.

In [ ]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' ' .join(pair)) # ("w1", "w2") -> "w1 w2"
    # Expresión regular utilizada para buscar el par más frecuente de caracteres
    # en el vocabucario"
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

Para probar la función utilizamos el par más frecuente de caracteres de `vocabulario`

In [ ]:
new_vocab = merge_vocab(best, vocabulario)
print(f"El par mas frequente: {best}")
print("Nuevo vocabulario:\n")
for word, freq in new_vocab.items():
    print(f"{word}: {freq}")

### **Get tokens**
La función `get_tokens` regresa los los tipos y su frecuencia del vocabulario del corpus.

In [ ]:
def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens

Los tipos de nuestro vocabulario serian

In [ ]:
tokens = get_tokens(vocabulario)

Ahora elaboramos una función auxiliar para guardar los tipos y su frecuencia en un archivo de texto

In [ ]:
def save_vocab(tokens, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(f"Tipo, Frecuencia\n")
        for token, freq in tokens.items():
            file.write(f"{token}, {freq}\n")

In [ ]:
save_vocab(tokens, 'vocab.txt')

### **Armando el algoritmo**

In [ ]:
def byte_pair_encoding(vocab, iter):
    vocab_aux = vocab
    for i in range(iter):
        pairs = get_stats(vocab_aux)
        if not pairs:
            break
        print(f"Iter {i+1}:")
        best = max(pairs, key=pairs.get)
        vocab_aux = merge_vocab(best, vocab_aux)
        tokens = get_tokens(vocab)
        print(f"Mejor par {best} \nNúmero de tokens: {len(tokens)}")
        print('============')
    return vocab_aux

## **Aplicandolo al Quijote**

In [ ]:
vocabulario = get_vocab('corpus/quijote.txt')

In [ ]:
save_vocab(get_tokens(vocabulario), 'vocabulario_inicial.txt')

**Número de iteraciones del algoritmo:**

In [ ]:
num_merges = 500

Utilizando el algoritmo

In [ ]:
vocabulario = byte_pair_encoding(vocabulario,num_merges)

**Guardamos nuestro vocabulario**

In [ ]:
save_vocab(get_tokens(vocabulario), 'vocab_quijote.txt')

## **Encoding and Decoding**

## **Conclusiones**